In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 pyspark-shell'

In [2]:
import pyspark
import os
import re

In [ ]:
def actualisation(base, batch):
    
    base = base.union(batch.na.drop() \
                           .orderBy('t', ascending=False) \
                           .dropDuplicates(subset = ['PhoneId']))
    
    base = base.orderBy('t', ascending=False) \
               .coalesce(1) \
               .dropDuplicates(subset = ['PhoneId'])
    
    return base

In [3]:
DATA_PATH = '../../data/madrid_sim'

In [4]:
files = [f for f in os.listdir(DATA_PATH) if (os.path.isfile(os.path.join(DATA_PATH, f)) and re.match('\d_MNO_MNO1.csv', f[-14:]))]

FileNotFoundError: [Errno 2] No such file or directory: '../../data/madrid_sim'

In [5]:
sc = pyspark.SparkContext()

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
sqlContext = pyspark.SQLContext(sc)
sc.addFile(os.path.join(DATA_PATH, files[0]))

In [8]:
from pyspark.sql.types import *

schema = StructType([StructField("t", IntegerType(), True), 
                     StructField("AntennaId", IntegerType(), True), 
                     StructField("EventCode", IntegerType(), True),
                     StructField("PhoneId", IntegerType(), True), 
                     StructField("x", FloatType(), True),
                     StructField("y", FloatType(), True),
                     StructField("TileId", IntegerType(), True)
                   ])

In [9]:
main_df = sqlContext.read.csv(pyspark.SparkFiles.get(files[0]), header=True, schema=schema)

In [10]:
main_df.show()

+---+---------+---------+-------+---+---+------+
|  t|AntennaId|EventCode|PhoneId|  x|  y|TileId|
+---+---------+---------+-------+---+---+------+
+---+---------+---------+-------+---+---+------+



In [ ]:
def actualisation(base, batch):
    
    base = base.union(batch.na.drop() \
                           .orderBy('t', ascending=False) \
                           .dropDuplicates(subset = ['PhoneId']))
    
    base = base.orderBy('t', ascending=False) \
               .coalesce(1) \
               .dropDuplicates(subset = ['PhoneId'])
    
    return base

df0 = actualisation(df0, df1)

tile_base = df0.groupBy('TileId').count()

In [10]:
from pykafka import KafkaClient
import json 
from datetime import datetime
import time
import uuid 
import pandas as pd

input_file= pd.read_csv('/home/cerezamo/projet_stream/simulator/mobile_data/kafka_minimal_df.csv')

client = KafkaClient(hosts="localhost:9092")
topic=client.topics['antennos']
producer = topic.get_sync_producer()

input_file = input_file[['t','PhoneId','x','y']]

input_file.PhoneId = input_file.PhoneId.astype(str)
input_file.x = input_file.x/1000
input_file.y = input_file.y/1000

def generate_checkpoint(input_file):
    for i in range(0,input_file.t.max()):
        input_file_topush = input_file[input_file.t==i]
        data = {'x' : [] , 'y' : [] , 'text' : []}
        
        j=0
        while j < len(input_file_topush):
            data['text'].append(input_file['PhoneId'][j])
            data['x'].append(input_file['x'][j]/1000)
            data['y'].append(input_file['y'][j]/1000)
            j+=1
            
        message = json.dumps(data)
        print(message)
        producer.produce(message.encode('ascii'))
        time.sleep(3)
        
generate_checkpoint(input_file)

In [ ]:
def generate_checkpoint(input_file):
    for i in range(0,input_file.t.max()):
        input_file_topush = input_file[input_file.t==i]
        message = input_file_topush.to_json(orient='records')
        print(message)
        producer.produce(message.encode('ascii'))
        time.sleep(3)
        
generate_checkpoint(input_file)

[{"t":0,"PhoneId":"296","x":600.995281445,"y":652.785571876},{"t":0,"PhoneId":"297","x":600.995281445,"y":652.785571876},{"t":0,"PhoneId":"289","x":600.384426337,"y":650.135919067},{"t":0,"PhoneId":"257","x":600.901554282,"y":651.774870814},{"t":0,"PhoneId":"259","x":600.203078702,"y":651.73008},{"t":0,"PhoneId":"230","x":601.129055069,"y":653.193950076},{"t":0,"PhoneId":"283","x":602.103592967,"y":649.742183146},{"t":0,"PhoneId":"284","x":602.103592967,"y":649.742183146},{"t":0,"PhoneId":"268","x":600.564455101,"y":652.729315347},{"t":0,"PhoneId":"241","x":601.462694847,"y":648.964107958},{"t":0,"PhoneId":"242","x":601.462694847,"y":648.964107958},{"t":0,"PhoneId":"234","x":601.514911596,"y":650.068348954},{"t":0,"PhoneId":"273","x":600.216462154,"y":649.458409534},{"t":0,"PhoneId":"291","x":600.50961435,"y":651.407658028},{"t":0,"PhoneId":"292","x":600.50961435,"y":651.407658028},{"t":0,"PhoneId":"270","x":600.008765087,"y":650.833277593},{"t":0,"PhoneId":"299","x":600.431357885,"y":

[{"t":5,"PhoneId":"242","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"241","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"230","x":600.660717662,"y":651.332288618},{"t":5,"PhoneId":"289","x":600.384426337,"y":650.135919067},{"t":5,"PhoneId":"257","x":600.242429502,"y":652.408177728},{"t":5,"PhoneId":"230","x":600.660717662,"y":651.332288618},{"t":5,"PhoneId":"284","x":601.326463487,"y":650.448298249},{"t":5,"PhoneId":"283","x":601.326463487,"y":650.448298249},{"t":5,"PhoneId":"253","x":600.091015093,"y":649.791594036},{"t":5,"PhoneId":"297","x":600.373529431,"y":653.390314852},{"t":5,"PhoneId":"296","x":600.373529431,"y":653.390314852},{"t":5,"PhoneId":"234","x":601.514911596,"y":650.068348954},{"t":5,"PhoneId":"257","x":600.242429502,"y":652.408177728},{"t":5,"PhoneId":"268","x":601.159144747,"y":650.051779337},{"t":5,"PhoneId":"242","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"241","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"273","x":600.19574499,"y"